In [2]:
#Bokeh plotting
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Span, Label, Arrow, NormalHead
from bokeh.models import HoverTool, tools, ColumnDataSource, CustomJS, Slider, BoxAnnotation
from bokeh.layouts import  column, row
from bokeh.palettes import Category20_18
import re
import glob, os
from astropy.io import fits
import urllib
#from urllib import urlretrieve
from pyraf import iraf
output_notebook()
iraf.noao.onedspec()
iraf.dataio()

Loading BokehJS ...

In [3]:
#Emission Lines
class line(object):
    def __init__(self,name,linecenter,regiontofit):
        self.name = name
        self.linecenter = linecenter
        self.regiontofit = regiontofit
        
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx, array[idx]

def isDigit(x):
    try:
        float(x)
        return True
    except ValueError:
        return False
    
def gaussian(x, mu, sig,core):
    return core*np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

#Emission Lines
class line(object):
    """Emission line use to plot and to fit"""
    def __init__(self,name,linecenter,regiontofit):
        self.name = name
        self.linecenter = linecenter
        self.regiontofit = regiontofit
        

class spec(object):
    """Spectra created from the url. Probably in the future better with the fiber
    , plate and MJD"""
    def __init__(self,url):
        self.url = url
        self.name = url.split('/')[-1]
        self.wave, self.flux, self.model, self.type = self.get_type()
       
            
    def get_type(self, verbose=False):
        """Get the type, flux, wavelenght and model from the SDSS TABLE. 
        It downloadst he fit if it doesnt exist. Had to do it like that becase it would waint until the download
        to call the other method"""
        #Download if it doesnt't exists. 
        if not os.path.isfile(self.name):
            file_name = self.name
            u = urllib.urlopen(self.url)
            f = open(file_name, 'wb')
            meta = u.info()
            file_size = int(meta.getheaders("Content-Length")[0])
            #print "Downloading: %s Bytes: %s" % (file_name, file_size)

            file_size_dl = 0
            block_sz = 8192
            while True:
                buffer = u.read(block_sz)
                if not buffer:
                    break

                file_size_dl += len(buffer)
                f.write(buffer)
                status = r"%10d  [%3.2f%%]" % (file_size_dl, file_size_dl * 100. / file_size)
                status = status + chr(8)*(len(status)+1)
                if verbose:
                    print status,

            f.close()
            #print('Downloaded '+ self.name)
        
        ob = fits.open(self.name)
        #Get data and save wavelenft in x, and flux on y
        dataob = ob[1].data
        x = 10**(dataob['loglam'])
        y2 = dataob['model']
        y = dataob['flux']
        typee = ob[2].data['SUBCLASS'][0]
        return x,y,y2, typee
    
    def get_iraffits(self, modelfit = True):
        """From the SDSS table creates a fits file to work with iraf"""
        #Which one to convert to iraf fts
        if modelfit:
            fluxormodel = self.model
        else:
            fluxormodel = self.flux
        #Name of iraf .txt file and create text file
        
        namenewfits = '{name}.txt'.format(name=self.name)
        with open(namenewfits,'w') as file:
            for x,y in zip(self.wave,fluxormodel):
                file.write('{}\t{}\n'.format(x,y))
                
        #Create fits file from text file interpolation to work wtih IRAF functions
        iraffitsname = 'iraf'+self.name
        if os.path.exists(iraffitsname+'.fits'):
            os.remove(iraffitsname+'.fits')
        iraf.rspectext(namenewfits,iraffitsname,dtype='interp')
        #Remove the text file
        os.remove(namenewfits)
        
    
    def fit_lines(self, dicoflines, errorestimate = True, verbose='No'):
        """Fit lines using gaussian fitport. It populates a diccionary with the lines
         and the fit of the lines. The parameter if a diccionary of lines objects. 
         The class is define before. Verbose can be yes or NO"""
        self.linesdicall = []
        errorparam = []
        if not os.path.exists('iraf'+self.name):
            self.get_iraffits()
            
        #Initialize files log and lines
        ! echo '' > fited.log

        for indexline,linesfit in enumerate(dicoflines):
            regionf = "{} {}".format(linesfit.regiontofit[0],linesfit.regiontofit[1])
            #wavelenght
            xlimns = [find_nearest(self.wave,i)[0] for i in linesfit.regiontofit   ]
            wavex = self.wave[xlimns[0]:xlimns[1]]
            lineszero = linesfit.linecenter
            ! echo '$lineszero' > lines.lines

            filename = 'iraf'+self.name
            #Error estimation iraf: http://stsdas.stsci.edu/cgi-bin/gethelp.cgi?fitprofs
            if errorestimate:        
                iraf.fitprofs(filename,pos='lines.lines', reg=regionf ,
                              fitbackground= 'yes', 
                              logfile='fited.log'
                              ,nerrsample='100',sigma0='4',invgain='4',verbose=verbose)
            else:
                iraf.fitprofs(filename,pos='lines.lines', reg=regionf ,
                              fitbackground= 'yes', 
                              logfile='fited.log', verbose=verbose)

            #Plotting the gaussian
            #Find in log file of fitprofs.  
            npattern = re.compile('[-\d.]+')
            npattern2 = re.compile('[-\d.E?]+') #Gets the exponentioals

            gparameters=[]
            with open('fited.log','r') as file:
                for lines in file:

                    if '(' not in lines:
                        temp = npattern2.findall(lines)
                        if 'INDEF' in lines:
                            gparameters.append(7*[0])
                    else:
                        errorparam.append(npattern2.findall(lines))

                    if len(temp) == 7 and all(isDigit(i) for i in temp) and 'INDEF' not in lines:
                        gparameters.append(temp)

            #gaussian
            if len(gparameters) > 0:
                gparamfinal = [ float(i) for i in gparameters[-1] ]
                centerg, contg, fluxg, eqwg, coreg, fwhmg, fwhml = gparamfinal
                yg = gaussian(wavex,centerg,fwhmg/2.3538,coreg) + contg


            if errorestimate == True:
                errorparamfinal = [ float(i) for i in errorparam[-1] ]
                #print(gparamfinal)
                #print(errorparamfinal)


                linesdic = {'linename':linesfit.name,
                             'center':centerg,
                              'EW': eqwg,
                              'EWerror':errorparamfinal[3],
                            'fluxg':fluxg,
                            'coregaus':coreg,
                            'fwgmgaus':fwhmg,
                            'contgaus':contg,
                            'gaussian':{'x':wavex,'y':yg}
                            }
            else:
                 linesdic = {'linename':linesfit.name,
                             'center':centerg,
                              'EW': eqwg,
                              'EWerror':0,
                            'fluxg':fluxg,
                            'coregaus':coreg,
                            'fwgmgaus':fwhmg,
                            'contgaus':contg,
                            'gaussian':{'x':wavex,'y':yg}
                            }
                
                
                
            self.linesdicall.append(linesdic)

        
        


In [4]:
#Define the new object
tryspec = spec('https://dr14.sdss.org/sas/dr14/sdss/spectro/redux/26/spectra/0266/spec-0266-51630-0015.fits')

In [7]:
diclines = {line('P15',8547,[8520,8560]),line('P14',8600,[8600-20,8600+20]),
           line('Halpha',6562,[6550,6575]),line('P16',8504,[8504-20,8504+20]),
            line('P13',8667,[8667-20,8667+20]),line('P12',8752,[8752-20,8752+20]),
           line('P11',8865,[8865-20,8865+20]),line('P10',9017,[9017-20,9017+20])}

dicofspectra = [];
filestoread = ['../listA1V.txt','../listB1Ve.txt',
              '../listF2V.txt','../listO.txt','../listG1V.txt',
              '../listK0V.txt','../listM0V.txt','../listM2V.txt']
limitonlines = 1
for files in filestoread:
    with open(files,'r') as f:
        linereads = f.readlines()
        for i in range(min(limitonlines, len(linereads))):
            lineread = linereads[i]
            #print(lineread)
            url = lineread.split(',')[1]
            #Strip Return a copy of the string with leading and trailing characters removed. 
            url = url.strip()
            tempspec = spec(url)
            dicofspectra.append(tempspec)
            
            
#Plot parameters
colors = 100*Category20_18 #+ Viridis8 + Dark2_8 + Paired8 + Set1_4
#Plot range
xr = (5000,8900)
yr = (0,40)

#Create the Bokeh Figure
pl =  figure(x_axis_label='Angstrom', y_axis_label='Y',title="Click on the desired stellar template to overplot", x_range=xr, y_range=yr
              ,active_drag='pan', active_scroll='wheel_zoom',
              plot_width=900, plot_height=1000
             )
#Tool to get wavelength
hover2 = HoverTool(
        tooltips=[
            ("(x,y)", "($x{1}, $y)"),
        ]
    )
#Add the tool
pl.add_tools(hover2)
#Start index of color
index = 0
fit = True


#Lines to fit
diclines = {line('P15',8547,[8520,8560]),line('P14',8600,[8600-20,8600+20]),
           line('Halpha',6562,[6550,6575]),line('P16',8504,[8504-20,8504+20]),
            line('P13',8667,[8667-20,8667+20]),line('P12',8752,[8752-20,8752+20]),
           line('P11',8865,[8865-20,8865+20]),line('P10',9017,[9017-20,9017+20])}



# Loop over spec object list:

for spectra in dicofspectra:

    # Plot the 
    t = pl.line(x=spectra.wave,y=spectra.model,color=colors[index], line_alpha=1.0, 
            line_width=4,legend=spectra.type+str(index),muted_alpha=0.,muted_color=colors[index])

    if fit:
        try:
            spectra.fit_lines(diclines, verbose='No', errorestimate=False)
        except:
            pass
        for i in spectra.linesdicall:
            pl.line(i['gaussian']['x'],i['gaussian']['y'],color='red', line_alpha=1.,
                line_width=5,legend=tryspec.name,muted_alpha=0.,muted_color=colors[index])

    index = index +1

pl.legend.location = "top_left"
pl.legend.click_policy="mute"

show(pl)  

            

/home/mmarcano/anaconda3/envs/iraf27/lib/python2.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in divide


In [ ]:
dicofspectra

In [36]:
filestoread = glob.glob('../listA*.txt')
filestoread

[]

In [37]:
pwd

u'/home/mmarcano/SDSSSpectra'